In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestRegressor

path = ""
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")

# One-hot-encode categorical variables
train['dataset'] = "train"
test['dataset'] = "test"
data = pd.concat([train,test], axis = 0)
categorical = ['property_type','room_type','bed_type','cancellation_policy','city']
data = pd.get_dummies(data, columns = categorical)



# Select only numeric data and impute missing values as 0
numerics = ['uint8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
train_x = data[data.dataset == "train"] \
    .select_dtypes(include=numerics) \
    .drop("log_price", axis = 1) \
    .fillna(0) \
    .values

test_x = data[data.dataset == "test"] \
    .select_dtypes(include=numerics) \
    .drop("log_price", axis = 1) \
    .fillna(0) \
    .values
    
train_y = data[data.dataset == "train"].log_price.values

# Train a Random Forest model with cross-validation

from sklearn.model_selection import KFold
cv_groups = KFold(n_splits=3)
regr = RandomForestRegressor(random_state = 0, n_estimators = 10)

for train_index, test_index in cv_groups.split(train_x):
    
    # Train the model using the training sets
    regr.fit(train_x[train_index], train_y[train_index])
    
    # Make predictions using the testing set
    pred_rf = regr.predict(train_x[test_index])
    
    # Calculate RMSE for current cross-validation split
    rmse = str(np.sqrt(np.mean((train_y[test_index] - pred_rf)**2)))
    
    print("RMSE for current split: " + rmse)
    
# Create submission file
regr.fit(train_x, train_y)
final_prediction = regr.predict(test_x)

submission = pd.DataFrame(np.column_stack([test.id, final_prediction]), columns = ['id','log_price'])
submission.to_csv("sample_submission.csv", index = False)


RMSE for current split: 0.4231951348239829
RMSE for current split: 0.4162644608016151
RMSE for current split: 0.42764534334464


In [65]:
print(data.columns)

Index(['accommodates', 'amenities', 'bathrooms', 'bedrooms', 'beds',
       'cleaning_fee', 'dataset', 'description', 'first_review',
       'host_has_profile_pic', 'host_identity_verified', 'host_response_rate',
       'host_since', 'id', 'instant_bookable', 'last_review', 'latitude',
       'log_price', 'longitude', 'name', 'neighbourhood', 'number_of_reviews',
       'review_scores_rating', 'thumbnail_url', 'zipcode',
       'property_type_Apartment', 'property_type_Bed & Breakfast',
       'property_type_Boat', 'property_type_Boutique hotel',
       'property_type_Bungalow', 'property_type_Cabin',
       'property_type_Camper/RV', 'property_type_Casa particular',
       'property_type_Castle', 'property_type_Cave', 'property_type_Chalet',
       'property_type_Condominium', 'property_type_Dorm',
       'property_type_Earth House', 'property_type_Guest suite',
       'property_type_Guesthouse', 'property_type_Hostel',
       'property_type_House', 'property_type_Hut', 'property_type

In [4]:
type(train["amenities"][0])

str

In [5]:
x = train["amenities"][0]
print(x)
print(type(x))

{"Wireless Internet","Air conditioning",Kitchen,Heating,"Family/kid friendly",Essentials,"Hair dryer",Iron,"translation missing: en.hosting_amenity_50"}
<class 'str'>


In [7]:
b = train["amenities"]
train["amenities"] = [x.split(",") for x in train["amenities"]]
print(b)

0        [{"Wireless Internet", "Air conditioning", Kit...
1        [{"Wireless Internet", "Air conditioning", Kit...
2        [{TV, "Cable TV", "Wireless Internet", "Air co...
3        [{TV, "Cable TV", Internet, "Wireless Internet...
4        [{TV, Internet, "Wireless Internet", "Air cond...
5        [{TV, "Wireless Internet", Heating, "Smoke det...
6        [{TV, Internet, "Wireless Internet", "Air cond...
7        [{TV, "Cable TV", "Wireless Internet", "Wheelc...
8        [{TV, "Cable TV", "Wireless Internet", "Pets l...
9        [{"Wireless Internet", "Air conditioning", Kit...
10       [{Internet, "Wireless Internet", "Air conditio...
11       [{TV, "Cable TV", Internet, "Wireless Internet...
12       [{TV, Internet, "Wireless Internet", Kitchen, ...
13       [{TV, "Cable TV", Internet, "Wireless Internet...
14       [{Kitchen, Heating, "Smoke detector", "Carbon ...
15       [{Internet, "Wireless Internet", "Air conditio...
16       [{TV, "Wireless Internet", Kitchen, "Pets allo.

In [8]:
#BREAKING OUT THE AMENITIIES INTO INDIVIDUAL COLUMN FLAGS

# print(train["amenities"][0][1])

# def check_it(input_str):
#     if "Air conditioning" in input_str:
#         return True
#     return False

# train.loc[:, 'air_cond_flag'] = train['amenities'][.apply(check_it)


In [34]:
path = ""
income_data = pd.read_csv(path + "ACS_16_5YR_S1903_with_ann.csv")

C:\Users\varbhatnagar\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1,3,4,59,60,83,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


AttributeError: 'Series' object has no attribute 'find'

In [9]:
path = ""
income_data = pd.read_csv(path + "ACS_16_5YR_S1903_with_ann.csv")
cleaned=income_data[['GEO.display-label', 'HC02_EST_VC02']]
# cleaned['zipcode'] = {x.replace('ZCTA5 ', '') for x in cleaned['GEO.display-label']}
cleaned["zipcode"]=cleaned['GEO.display-label'].str.slice(6,13)
cleaned.drop(cleaned.index[0], inplace=True)
cleaned.drop(columns=['GEO.display-label'], inplace=True)
data.merge(cleaned, on='zipcode', how='left')

C:\Users\varbhatnagar\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1,3,4,59,60,83,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\varbhatnagar\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\varbhatnagar\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\varbh

,accommodates,amenities,bathrooms,bedrooms,beds,cleaning_fee,dataset,description,first_review,host_has_profile_pic,...,cancellation_policy_strict,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60,city_Boston,city_Chicago,city_DC,city_LA,city_NYC,city_SF,HC02_EST_VC02
0,3,"{""Wireless Internet"",""Air conditioning"",Kitche...",1.0,1.0,1.0,True,train,"Beautiful, sunlit brownstone 1-bedroom in the ...",2016-06-18,t,...,1,0,0,0,0,0,0,1,0,109472
1,7,"{""Wireless Internet"",""Air conditioning"",Kitche...",1.0,3.0,3.0,True,train,Enjoy travelling during your stay in Manhattan...,2017-08-05,t,...,1,0,0,0,0,0,0,1,0,94477
2,5,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",1.0,1.0,3.0,True,train,The Oasis comes complete with a full backyard ...,2017-04-30,t,...,0,0,0,0,0,0,0,1,0,42754
3,4,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",1.0,2.0,2.0,True,train,This light-filled home-away-from-home is super...,NaN,t,...,0,0,0,0,0,0,0,0,1,NaN
4,2,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.0,0.0,1.0,True,train,"Cool, cozy, and comfortable studio located in ...",2015-05-12,t,...,0,0,0,0,0,1,0,0,0,97563
5,2,"{TV,""Wireless Internet"",Heating,""Smoke detecto...",1.0,1.0,1.0,True,train,Beautiful private room overlooking scenic view...,2017-08-27,t,...,1,0,0,0,0,0,0,0,1,111083
6,3,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.0,1.0,1.0,True,train,Warm and cozy studio with full kitchen and bat...,2017-03-10,t,...,0,0,0,0,0,0,1,0,0,100842
7,2,"{TV,""Cable TV"",""Wireless Internet"",""Wheelchair...",1.0,1.0,1.0,True,train,Arguably the best location (and safest) in dow...,2016-12-16,t,...,0,0,0,0,0,0,1,0,0,30463
8,2,"{TV,""Cable TV"",""Wireless Internet"",""Pets live ...",1.0,1.0,1.0,True,train,Garden Studio with private entrance from the s...,2016-02-13,t,...,0,0,0,0,0,0,0,0,1,77280
9,2,"{""Wireless Internet"",""Air conditioning"",Kitche...",1.0,1.0,1.0,True,train,"Quiet community. Close to supermarkets,restaur...",2017-04-03,t,...,0,0,0,0,0,0,1,0,0,61740


In [7]:
import lda
import lda.datasets
X = lda.datasets.load_reuters()
vocab = lda.datasets.load_reuters_vocab()
titles = lda.datasets.load_reuters_titles()